<a href="https://www.kaggle.com/code/roberttarus/toronto-bikeshare-dataset-cleaning-and-using-joins?scriptVersionId=166324784" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

The Toronto Parking Authority manages the Bike Share Toronto program, a form of local mass transit and an enjoyable way to travel the city by bicycle. Bike Share stations are located in Toronto, East York, Scarborough, North York, York and Etobicoke.

In our data analysis, we're diving into the world of bikeshare data, weather information, and station details. When we put these pieces together, we get a richer picture of how people use bikes in different conditions and locations.

**Cleaning the Bikeshare Data:**

Cleaning this data means we're making sure it's accurate and reliable. This way, our insights can be trusted, and decisions can be based on solid information.

**Adding Stations:**

Stations are like the hubs of the bikeshare network. Combining bikeshare data with station info helps us understand which stations are popular, how many bikes they can hold, and how efficiently the system is running. It's like putting a map to our data, showing where the action is happening.

**Connecting with Weather Data:**

Ever wondered how weather affects our daily activities, including bike rides? That's what we're exploring by merging bikeshare data with weather conditions. Does rain or sunshine impact how many people use bikes? By joining these datasets, we're getting a clear picture of how weather plays a role in bike-sharing trends.

**Goal of the analysis**

Ultimately, we're doing all of this to uncover insights that matter. Whether it's improving the user experience, managing stations better, or adapting to weather patterns, our goal is to make the bikeshare system more efficient and enjoyable for everyone involved.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
import json
import glob
warnings.filterwarnings("ignore")
warnings.resetwarnings()
%matplotlib inline

In [2]:
directory_path = '/kaggle/input/toronto-bikeshare-2023'

# Get a list of all CSV files in the directory and sort them
csv_files = sorted([f for f in os.listdir(directory_path) if f.endswith('.csv')])

dfs = []

# Looping through each CSV file, read and append to the list of DataFrames
for csv in csv_files:
    try:
        df = pd.read_csv(os.path.join(directory_path, csv), encoding='utf-8')
    except UnicodeDecodeError:
        # If utf-8 fails, try 'latin-1' encoding
        df = pd.read_csv(os.path.join(directory_path, csv), encoding='latin-1')
    dfs.append(df)

# Concatenating the list of DataFrames into a single DataFrame
bikes_df = pd.concat(dfs, ignore_index=True)



In [3]:
#Checking the data types in the dataframe
bikes_df.dtypes 

Trip Id                 int64
Trip  Duration          int64
Start Station Id        int64
Start Time             object
Start Station Name     object
End Station Id        float64
End Time               object
End Station Name       object
Bike Id                 int64
User Type              object
dtype: object

In [4]:
# Renaming columns
bikes_df = bikes_df.rename(columns={
    'Trip Id': 'trip_id',
    'Trip  Duration': 'trip_duration',
    'Start Station Id': 'start_station_id',
    'Start Time': 'start_time',
    'Start Station Name': 'start_station_name',
    'End Station Id': 'end_station_id',
    'End Time': 'end_time',
    'End Station Name': 'end_station_name',
    'Bike Id': 'bike_id',
    'User Type': 'user_type'
})

In [5]:
bikes_df.dtypes

trip_id                 int64
trip_duration           int64
start_station_id        int64
start_time             object
start_station_name     object
end_station_id        float64
end_time               object
end_station_name       object
bike_id                 int64
user_type              object
dtype: object

In [6]:
#Converting the end_station_column datatype to string.
bikes_df['end_station_id'] = bikes_df['end_station_id'].astype(str)

In [7]:
unique_values = bikes_df['end_station_id'].unique()

# Print the unique values
print("Unique values in 'end_station_id':", unique_values)

Unique values in 'end_station_id': ['7703.0' '7533.0' '7076.0' '7685.0' '7061.0' '7033.0' '7521.0' '7760.0'
 '7075.0' '7014.0' '7161.0' '7164.0' '7098.0' '7100.0' '7154.0' '7167.0'
 '7475.0' '7430.0' '7026.0' '7526.0' '7368.0' '7493.0' '7502.0' '7329.0'
 '7261.0' '7264.0' '7682.0' '7367.0' '7006.0' '7469.0' '7418.0' '7043.0'
 '7030.0' '7202.0' '7398.0' '7057.0' '7240.0' '7044.0' '7337.0' '7120.0'
 '7259.0' '7079.0' '7529.0' '7149.0' '7078.0' '7238.0' '7356.0' '7055.0'
 '7453.0' '7270.0' '7457.0' '7719.0' '7225.0' '7168.0' '7542.0' '7311.0'
 '7650.0' '7191.0' '7048.0' '7361.0' '7108.0' '7198.0' '7523.0' '7114.0'
 '7207.0' '7346.0' '7001.0' '7066.0' '7320.0' '7226.0' '7566.0' '7085.0'
 '7197.0' '7170.0' '7272.0' '7382.0' '7028.0' '7077.0' '7298.0' '7109.0'
 '7022.0' '7136.0' '7309.0' '7178.0' '7162.0' '7414.0' '7169.0' '7021.0'
 '7209.0' '7483.0' '7281.0' '7041.0' '7020.0' '7646.0' '7472.0' '7236.0'
 '7399.0' '7534.0' '7121.0' '7257.0' '7205.0' '7222.0' '7688.0' '7143.0'
 '7059.0' '7054.

In [8]:
unique_values = bikes_df['end_station_name'].unique()
print("Unique values in 'end_station_name':", unique_values)

Unique values in 'end_station_name': [nan 'Housey St / Dan Leckie Way' 'York St / Queens Quay W'
 'Dalton Rd / Bloor St W' 'Union Station' 'Emerson Ave / Bloor St W'
 'Queens Quay W / Dan Leckie Way'
 'Sherbourne St / Carlton St (Allan Gardens)' 'Beverley St / College St'
 'Gould St / Yonge St (Ryerson University)'
 'Riverdale Park South (Broadview Ave)' 'Dundas St E / Regent Park Blvd'
 'Bathurst Subway Station' 'Queen St E / George St (Moss Park)'
 'Jarvis St / Richmond St E' 'Marilyn Bell Park Tennis Court'
 'Bay St / St. Joseph St' 'Bartlett Parkette'
 'Lisgar St / Dundas St SMART' 'Sackville St / Eastern Ave - SMART'
 'University Ave / College St (East)' 'Crawford St / Queen St W'
 'Queens Quay E / Lower Sherbourne St' 'Bloor St E / Huntley St - SMART'
 'Alma Ave / Gladstone Ave SMART' 'Bay St / College St (East Side)'
 'Wellington St W / York St' 'College Park - Yonge St Entrance'
 'Queens Quay W / Lower Simcoe St' 'Bay St / Wellesley St W'
 'Queen St W / York St (City Hall)' 'Yo

In [9]:
#Replacing 'nan' values with 'unknown'
bikes_df = bikes_df.replace({'nan': 'unknown'})

In [10]:
specific_value = 'nan'
found_rows = bikes_df[bikes_df['end_station_id'] == specific_value]

# Print the rows where the nan values are found
print(f"\nRows where 'end_station_id' is {specific_value}:\n", found_rows)


Rows where 'end_station_id' is nan:
 Empty DataFrame
Columns: [trip_id, trip_duration, start_station_id, start_time, start_station_name, end_station_id, end_time, end_station_name, bike_id, user_type]
Index: []


In [11]:
#Converting datatypes and columns to string.
bikes_df['start_station_id'] = bikes_df['start_station_id'].astype(str)
bikes_df['trip_id'] = bikes_df['trip_id'].astype(str)
bikes_df['bike_id'] = bikes_df['bike_id'].astype(str)

In [12]:
#Preview of the dataset
bikes_df.head()

,trip_id,trip_duration,start_station_id,start_time,start_station_name,end_station_id,end_time,end_station_name,bike_id,user_type
0,20148784,840,7022,01/01/2023 00:00,Simcoe St / Queen St W,7703.0,01/01/2023 00:14,NaN,1770,Casual Member
1,20148785,722,7399,01/01/2023 00:01,Lower Jarvis / Queens Quay E,7533.0,01/01/2023 00:13,Housey St / Dan Leckie Way,6400,Casual Member
2,20148786,1054,7269,01/01/2023 00:02,Toronto Eaton Centre (Yonge St),7076.0,01/01/2023 00:20,York St / Queens Quay W,4068,Annual Member
3,20148790,1329,7721,01/01/2023 00:04,NaN,7685.0,01/01/2023 00:26,NaN,427,Casual Member
4,20148791,1291,7721,01/01/2023 00:04,NaN,7685.0,01/01/2023 00:26,NaN,1840,Casual Member


# Checking Null Values in each column

In [13]:
#Checking to see if the dataset has null values on each column
null_values = bikes_df['trip_id'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'trip_id': {null_values}")

null_values = bikes_df['trip_duration'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'trip_duration': {null_values}")

null_values = bikes_df['start_station_id'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'start_station_id': {null_values}")

null_values = bikes_df['end_station_id'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'end_station_id': {null_values}")

null_values = bikes_df['start_station_name'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'start_station_name': {null_values}")

null_values = bikes_df['end_station_name'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'end_station_name': {null_values}")

null_values = bikes_df['bike_id'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'bike_id': {null_values}")

null_values = bikes_df['start_time'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'start_time': {null_values}")

null_values = bikes_df['end_time'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'end_time': {null_values}")

null_values = bikes_df['user_type'].isnull().sum()

# Print the number of null values
print(f"Number of null values in 'user_type': {null_values}")

Number of null values in 'trip_id': 0
Number of null values in 'trip_duration': 0
Number of null values in 'start_station_id': 0
Number of null values in 'end_station_id': 0
Number of null values in 'start_station_name': 595075
Number of null values in 'end_station_name': 598563
Number of null values in 'bike_id': 0
Number of null values in 'start_time': 0
Number of null values in 'end_time': 0
Number of null values in 'user_type': 0


# **Replacing Null Values.**

In [14]:
bikes_df['start_station_name'] = bikes_df['start_station_name'].fillna('unknown')
bikes_df['end_station_id'] = bikes_df['end_station_id'].fillna('uknown')
bikes_df['end_station_name'] = bikes_df['end_station_name'].fillna('uknown')

In [15]:
bikes_df.head()

,trip_id,trip_duration,start_station_id,start_time,start_station_name,end_station_id,end_time,end_station_name,bike_id,user_type
0,20148784,840,7022,01/01/2023 00:00,Simcoe St / Queen St W,7703.0,01/01/2023 00:14,uknown,1770,Casual Member
1,20148785,722,7399,01/01/2023 00:01,Lower Jarvis / Queens Quay E,7533.0,01/01/2023 00:13,Housey St / Dan Leckie Way,6400,Casual Member
2,20148786,1054,7269,01/01/2023 00:02,Toronto Eaton Centre (Yonge St),7076.0,01/01/2023 00:20,York St / Queens Quay W,4068,Annual Member
3,20148790,1329,7721,01/01/2023 00:04,unknown,7685.0,01/01/2023 00:26,uknown,427,Casual Member
4,20148791,1291,7721,01/01/2023 00:04,unknown,7685.0,01/01/2023 00:26,uknown,1840,Casual Member


In [16]:
# Converting 'trip_duration' to numeric
bikes_df['trip_duration_seconds'] = pd.to_numeric(bikes_df['trip_duration'], errors='coerce')

# Converting 'end_time' and 'start_time' columns
bikes_df['start_time'] = pd.to_datetime(bikes_df['start_time'], format='%m/%d/%Y %H:%M', errors='coerce')
bikes_df['end_time'] = bikes_df['start_time'] + pd.to_timedelta(bikes_df['trip_duration_seconds'], unit='s')

# Splitting 'start_time' into 'start_date' and 'start_time' columns
bikes_df['start_date'] = bikes_df['start_time'].dt.date
bikes_df['start_time'] = bikes_df['start_time'].dt.time

# Splitting 'end_time' into 'end_date' and 'end_time' columns
bikes_df['end_date'] = bikes_df['end_time'].dt.date
bikes_df['end_time'] = bikes_df['end_time'].dt.time

# Reorganizing columns
bikes_df = bikes_df[['trip_id', 'trip_duration_seconds', 'start_station_id', 'start_station_name', 'start_time', 'start_date', 'end_station_id', 'end_station_name', 'end_time', 'end_date', 'bike_id', 'user_type']]


In [17]:
#Converting end_station_id to string and stripping the . and 0.
bikes_df['end_station_id'] = bikes_df['end_station_id'].astype(str).apply(lambda x: x.rstrip('.0'))

In [18]:
bikes_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,uknown,00:14:00,2023-01-01,1770,Casual Member
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member
3,20148790,1329,7721,unknown,00:04:00,2023-01-01,7685,uknown,00:26:09,2023-01-01,427,Casual Member
4,20148791,1291,7721,unknown,00:04:00,2023-01-01,7685,uknown,00:25:31,2023-01-01,1840,Casual Member


In [19]:
bikes_df.dtypes

trip_id                  object
trip_duration_seconds     int64
start_station_id         object
start_station_name       object
start_time               object
start_date               object
end_station_id           object
end_station_name         object
end_time                 object
end_date                 object
bike_id                  object
user_type                object
dtype: object

In [20]:
#Converting the start_date and end_date to date format.
bikes_df['start_date'] = pd.to_datetime(bikes_df['start_date'], format='%Y-%m-%d', errors='coerce')
bikes_df['end_date'] = pd.to_datetime(bikes_df['end_date'], format='%Y-%m-%d', errors='coerce')

In [21]:
bikes_df.dtypes

trip_id                          object
trip_duration_seconds             int64
start_station_id                 object
start_station_name               object
start_time                       object
start_date               datetime64[ns]
end_station_id                   object
end_station_name                 object
end_time                         object
end_date                 datetime64[ns]
bike_id                          object
user_type                        object
dtype: object

In [22]:
#Counting the number of unique values in each column
bikes_df.nunique()

trip_id                  5713141
trip_duration_seconds      22922
start_station_id             814
start_station_name           594
start_time                  1440
start_date                   365
end_station_id               816
end_station_name             593
end_time                   86331
end_date                     371
bike_id                     7446
user_type                      2
dtype: int64

In [23]:
#Checking for missing values in each column
bikes_df.isna().sum()

trip_id                  0
trip_duration_seconds    0
start_station_id         0
start_station_name       0
start_time               0
start_date               0
end_station_id           0
end_station_name         0
end_time                 0
end_date                 0
bike_id                  0
user_type                0
dtype: int64

In [24]:
#Specifying the column name to filter
column_name = 'start_station_name'

#Creating a subset of the dataframe to get rows with 'unknown' values
rows_with_unknown = bikes_df[bikes_df['start_station_name'] == 'unknown']

# Printing the rows with the value 'unknown' in the specified column
print(rows_with_unknown)

          trip_id  trip_duration_seconds start_station_id start_station_name  \
3        20148790                   1329             7721            unknown   
4        20148791                   1291             7721            unknown   
6        20148796                    786             7713            unknown   
10       20148801                    667             7698            unknown   
15       20148807                    764             7713            unknown   
...           ...                    ...              ...                ...   
5713113  26661192                    554             7720            unknown   
5713115  26663642                     95             7707            unknown   
5713127  26678261                      0             7719            unknown   
5713132  26679155                      0             7802            unknown   
5713137  26681858                      0             7770            unknown   

        start_time start_date end_stati

There are 595,075 rows that have 'unknown' values in the start_station_name column

In [25]:
#Specifying the column to filter
column_name = 'end_station_name'

#Checking the rows with 'uknown' values
rows_with_unknown = bikes_df[bikes_df['end_station_name'] == 'uknown']

# Printing the rows with the value 'unknown' in the specified column
print(rows_with_unknown)

          trip_id  trip_duration_seconds start_station_id  \
0        20148784                    840             7022   
3        20148790                   1329             7721   
4        20148791                   1291             7721   
11       20148802                   1644             7514   
14       20148806                   1477             7514   
...           ...                    ...              ...   
5713136  26680335                      0             7197   
5713137  26681858                      0             7770   
5713138  26682252                      0             7537   
5713139  26682255                      0             7537   
5713140  26682698                   3431             7411   

                                start_station_name start_time start_date  \
0                           Simcoe St / Queen St W   00:00:00 2023-01-01   
3                                          unknown   00:04:00 2023-01-01   
4                                      

There are 598,563 rows of 'uknown' value in the end_station_name column

In [26]:
#Format to three decimal places
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Summary statistics of bikes_df
bikes_df.describe()

,trip_duration_seconds,start_date,end_date
count,5713141.000,5713141,5713141
mean,1079.395,2023-07-22 05:57:15.351797760,2023-07-22 06:06:33.119373056
min,0.000,2023-01-01 00:00:00,2023-01-01 00:00:00
25%,421.000,2023-05-27 00:00:00,2023-05-27 00:00:00
50%,687.000,2023-07-28 00:00:00,2023-07-28 00:00:00
75%,1120.000,2023-09-23 00:00:00,2023-09-23 00:00:00
max,3980837.000,2023-12-31 00:00:00,2024-01-07 00:00:00
std,8797.974,NaN,NaN


In [27]:
# Filtering the bikes_df DataFrame to include only rows where the trip duration is 60 seconds or more
# This filtering helps remove very short trips

bikes_df = bikes_df[bikes_df['trip_duration_seconds'] >= 60]

In [28]:
#Summary statistics to three decimal places.
pd.set_option('display.float_format', lambda x: '%.3f' % x)
bikes_df.describe()

,trip_duration_seconds,start_date,end_date
count,5701110.000,5701110,5701110
mean,1081.655,2023-07-22 05:44:37.632250368,2023-07-22 05:53:56.576877568
min,60.000,2023-01-01 00:00:00,2023-01-01 00:00:00
25%,423.000,2023-05-27 00:00:00,2023-05-27 00:00:00
50%,689.000,2023-07-28 00:00:00,2023-07-28 00:00:00
75%,1121.000,2023-09-23 00:00:00,2023-09-23 00:00:00
max,3980837.000,2023-12-31 00:00:00,2024-01-07 00:00:00
std,8807.115,NaN,NaN


# Looking up the missing station names

In [29]:
# Making an HTTP GET request to the specified URL to retrieve station information
response = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')

# Converting the response content from JSON format to extract station data selecting specific columns
bikeshare_stations = pd.DataFrame(json.loads(response.content)['data']['stations'])[[
   'station_id', 'name', 'lat', 'lon', 'capacity'
   ]].astype({'station_id': 'str',})

#Displaying the first five rows
bikeshare_stations.head()

,station_id,name,lat,lon,capacity
0,7000,Fort York Blvd / Capreol Ct,43.640,-79.396,35
1,7001,Wellesley Station Green P,43.665,-79.384,23
2,7002,St. George St / Bloor St W,43.667,-79.399,19
3,7003,Madison Ave / Bloor St W,43.667,-79.403,15
4,7005,King St W / York St,43.648,-79.383,23


In [30]:
#Checking data types of each column
bikeshare_stations.dtypes

station_id     object
name           object
lat           float64
lon           float64
capacity        int64
dtype: object

In [31]:
# Identifying unique station IDs associated with 'unknown' start and end stations
unknown_start_station_ids = bikes_df['start_station_id'][bikes_df['start_station_name'] == 'unknown'].unique()
unknown_end_station_ids = bikes_df['end_station_id'][bikes_df['end_station_name'] == 'unknown'].unique()

# Filtering the bikeshare_stations_df DataFrame to find the IDs and names of stations with 'unknown' IDs
unknown_start_stations = bikeshare_stations[bikeshare_stations['station_id'].isin(unknown_start_station_ids)][['station_id', 'name']]
unknown_end_stations = bikeshare_stations[bikeshare_stations['station_id'].isin(unknown_end_station_ids)][['station_id', 'name']]

print("Unknown start stations:")
print(unknown_start_stations)

print("\nUnknown end stations:")
print(unknown_end_stations)

Unknown start stations:
    station_id                                name
562       7682            Bathurst St / Front St W
563       7684          Bay St / Harbour St (East)
564       7685                King St W / Brant St
565       7686     Lower Jarvis St / The Esplanade
566       7687  Bloor St W / Gladstone Ave - SMART
..         ...                                 ...
776       7914       York St / Wellington St W (2)
777       7915                Rogers Rd / Watt Ave
778       7916          University Ave / Pearl St 
779       7917          King St W / University Ave
780       7918               Albion Rd/ Arcot Blvd

[219 rows x 2 columns]

Unknown end stations:
Empty DataFrame
Columns: [station_id, name]
Index: []


In [32]:
# Identifying unique station IDs associated with 'unknown' start and end stations names
unknown_start_station_ids = bikes_df['start_station_id'][bikes_df['start_station_name'] == 'unknown'].unique()
unknown_end_station_ids = bikes_df['end_station_id'][bikes_df['end_station_name'] == 'unknown'].unique()

# Filtering the bikeshare_stations_df DataFrame to find the IDs and names of stations with 'unknown' IDs
unknown_start_stations = bikeshare_stations[bikeshare_stations['station_id'].isin(unknown_start_station_ids)][['station_id', 'name']]
unknown_end_stations = bikeshare_stations[bikeshare_stations['station_id'].isin(unknown_end_station_ids)][['station_id', 'name']]

# Replacing 'unknown' start station names in bikes_df
for index, row in unknown_start_stations.iterrows():
    bikes_df.loc[bikes_df['start_station_id'] == row['station_id'], 'start_station_name'] = row['name']

# Replacing 'unknown' end station names in bikes_df
for index, row in unknown_end_stations.iterrows():
    bikes_df.loc[bikes_df['end_station_id'] == row['station_id'], 'end_station_name'] = row['name']

# Verifying the changes
bikes_df.head()


,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,uknown,00:14:00,2023-01-01,1770,Casual Member
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,uknown,00:26:09,2023-01-01,427,Casual Member
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,uknown,00:25:31,2023-01-01,1840,Casual Member


In [33]:
#Filtering column name from dataframe
column_name = 'end_station_name'

# Filter rows where the 'end_station_name' column has the value 'unknown'
rows_with_unknown = bikes_df[bikes_df[column_name] == 'uknown']

# Printing the rows with the value 'unknown' in the specified column
print(rows_with_unknown)

          trip_id  trip_duration_seconds start_station_id  \
0        20148784                    840             7022   
3        20148790                   1329             7721   
4        20148791                   1291             7721   
11       20148802                   1644             7514   
14       20148806                   1477             7514   
...           ...                    ...              ...   
5713116  26664175                   1200             7055   
5713119  26667491                   9745             7016   
5713121  26670570                    868             7069   
5713122  26673283                    759             7543   
5713140  26682698                   3431             7411   

                                start_station_name start_time start_date  \
0                           Simcoe St / Queen St W   00:00:00 2023-01-01   
3                            Widmer St / King St W   00:04:00 2023-01-01   
4                            Widmer St 

We still have 99,025 rows that don't have their station names in the end_station_name column

In [34]:
#Inspecting the first 30 rows
rows_with_unknown.head(30)

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,uknown,00:14:00,2023-01-01,1770,Casual Member
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,uknown,00:26:09,2023-01-01,427,Casual Member
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,uknown,00:25:31,2023-01-01,1840,Casual Member
11,20148802,1644,7514,Humber Bay Shores Park / Marine Parade Dr,00:08:00,2023-01-01,776,uknown,00:35:24,2023-01-01,2337,Casual Member
14,20148806,1477,7514,Humber Bay Shores Park / Marine Parade Dr,00:10:00,2023-01-01,776,uknown,00:34:37,2023-01-01,2473,Casual Member
35,20148828,430,7533,Housey St / Dan Leckie Way,00:20:00,2023-01-01,7682,uknown,00:27:10,2023-01-01,6400,Annual Member
51,20148848,298,7000,Fort York Blvd / Capreol Ct,00:24:00,2023-01-01,7682,uknown,00:28:58,2023-01-01,401,Casual Member
66,20148864,749,7682,Bathurst St / Front St W,00:27:00,2023-01-01,7719,uknown,00:39:29,2023-01-01,6400,Annual Member
180,20148989,252,7223,Parkside Dr / Bloor St W - SMART,00:54:00,2023-01-01,7688,uknown,00:58:12,2023-01-01,4067,Casual Member
192,20149006,1080,7072,Fleet St / Bathurst St,00:57:00,2023-01-01,7684,uknown,01:15:00,2023-01-01,4933,Casual Member


In [35]:
# Filter rows where end_station_name is 'unknown'
unknown_end_stations = bikes_df[bikes_df['end_station_name'] == 'uknown']

# Get unique end_station_ids from the filtered DataFrame
unique_unknown_end_station_ids = unknown_end_stations['end_station_id'].unique()

# Display the result
print(unique_unknown_end_station_ids)

['7703' '7685' '776' '7682' '7719' '7688' '7684' '7721' '7713' '772'
 '7716' '7704' '7746' '7693' '7754' '7706' '7709' '7689' '7717' '7711'
 '7753' '7728' '7705' '7752' '7707' '7724' '77' '7692' '7757' '7738'
 '7701' '771' '7749' '7702' '7686' '7683' '7687' '7729' '7695' '773'
 '7694' '7718' '7744' '7732' '7708' '7712' '7699' '7715' '774' '7691'
 '7731' '7725' '775' '7748' '769' '7745' '7751' '7747' '7741' '7736'
 '7696' '7698' '7755' '7742' 'unknown' '7743' '7756' '7734' '7733' '7758'
 '7739' '7761' '7737' '7762' '7759' '7766' '7765' '7735' '7768' '7769'
 '7767' '777' '7771' '7772' '7773' '7777' '7778' '7774' '7775' '7779'
 '7776' '778' '7781' '7783' '7787' '7784' '7782' '7785' '7788' '779'
 '7791' '7723' '7792' '7794' '7795' '7796' '7798' '7799' '78' '7801'
 '7802' '7763' '7804' '7803' '7806' '7805' '7808' '7809' '7807' '781'
 '7811' '7812' '7815' '7814' '7813' '7816' '7817' '7818' '7819' '782'
 '7789' '7821' '7822' '7823' '7824' '7825' '7826' '7827' '7829' '783'
 '7828' '7831' '7832

In [36]:
# Filter bikeshare_stations for missing station IDs
missing_station_info = bikeshare_stations[bikeshare_stations['station_id'].isin(unique_unknown_end_station_ids)]

# Display the result
print(missing_station_info[['station_id', 'name']])


    station_id                                name
562       7682            Bathurst St / Front St W
563       7684          Bay St / Harbour St (East)
564       7685                King St W / Brant St
565       7686     Lower Jarvis St / The Esplanade
566       7687  Bloor St W / Gladstone Ave - SMART
..         ...                                 ...
775       7913        St. George St / Willcocks St
776       7914       York St / Wellington St W (2)
778       7916          University Ave / Pearl St 
779       7917          King St W / University Ave
780       7918               Albion Rd/ Arcot Blvd

[196 rows x 2 columns]


In [37]:
# Creating a dictionary mapping station_id to station_name for missing station information
station_id_to_name = dict(zip(missing_station_info['station_id'], missing_station_info['name']))

# Using map function to replace 'uknown' values in bikes_df
bikes_df['end_station_name'] = bikes_df['end_station_id'].map(station_id_to_name).combine_first(bikes_df['end_station_name'])

# Displaying the first five rows of the updated DataFrame
bikes_df.head()


,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,1770,Casual Member
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,427,Casual Member
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,1840,Casual Member


In [38]:
column_name = 'end_station_name'

rows_with_unknown = bikes_df[bikes_df['end_station_name'] == 'uknown']

# Print or view the rows with the value 'unknown' in the specified column
print(rows_with_unknown)

          trip_id  trip_duration_seconds start_station_id  \
11       20148802                   1644             7514   
14       20148806                   1477             7514   
242      20149060                    473             7417   
345      20149170                    130             7720   
399      20149226                   1216             7287   
...           ...                    ...              ...   
5713116  26664175                   1200             7055   
5713119  26667491                   9745             7016   
5713121  26670570                    868             7069   
5713122  26673283                    759             7543   
5713140  26682698                   3431             7411   

                                start_station_name start_time start_date  \
11       Humber Bay Shores Park / Marine Parade Dr   00:08:00 2023-01-01   
14       Humber Bay Shores Park / Marine Parade Dr   00:10:00 2023-01-01   
242                          King St W 

**Dropping rows with issues that cannot be fixed

* Reason: The 99025 rows have data quality issues that cannot be resolved, and retaining them may impact the accuracy and reliability of our analysis.**



In [39]:
# Drop rows where end_station_id is unknown
bikes_df = bikes_df[bikes_df['end_station_id'] != 'uknown']

# Reset the index if needed
bikes_df.reset_index(drop=True, inplace=True)

In [40]:
column_name = 'end_station_name'

rows_with_unknown = bikes_df[bikes_df['end_station_name'] == 'uknown']

# Print or view the rows with the value 'unknown' in the specified column
print(rows_with_unknown)

          trip_id  trip_duration_seconds start_station_id  \
11       20148802                   1644             7514   
14       20148806                   1477             7514   
242      20149060                    473             7417   
345      20149170                    130             7720   
399      20149226                   1216             7287   
...           ...                    ...              ...   
5701105  26664175                   1200             7055   
5701106  26667491                   9745             7016   
5701107  26670570                    868             7069   
5701108  26673283                    759             7543   
5701109  26682698                   3431             7411   

                                start_station_name start_time start_date  \
11       Humber Bay Shores Park / Marine Parade Dr   00:08:00 2023-01-01   
14       Humber Bay Shores Park / Marine Parade Dr   00:10:00 2023-01-01   
242                          King St W 

In [41]:
# Filter rows where end_station_name is 'unknown'
unknown_end_stations = bikes_df[bikes_df['end_station_name'] == 'uknown']

# Get unique end_station_ids from the filtered DataFrame
unique_unknown_end_station_ids = unknown_end_stations['end_station_id'].unique()

# Display the result
print(unique_unknown_end_station_ids)

['776' '772' '7689' '77' '771' '7683' '773' '774' '775' '769' 'unknown'
 '777' '778' '779' '7723' '7792' '78' '7763' '781' '782' '783' '784' '785'
 '786' '787' '79' '788' '789' '791']


In [42]:
# Drop rows with any column containing 'uknown' values
bikes_df_cleaned = bikes_df.replace('uknown', pd.NA).dropna()

# Reset the index if needed
bikes_df_cleaned.reset_index(drop=True, inplace=True)

In [43]:
#Inspecting the first five rows after dropping the rows we could not fix.
bikes_df_cleaned.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,1770,Casual Member
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,427,Casual Member
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,1840,Casual Member


In [44]:
# Check if 'unknown' values exist in the entire DataFrame
unknown_values_exist = (bikes_df_cleaned == 'uknown').any().any()

# Display the result
print(f"Do 'unknown' values exist in the DataFrame? {unknown_values_exist}")

Do 'unknown' values exist in the DataFrame? False


In [45]:
# Check if 'unknown' values exist in the entire DataFrame
unknown_values_exist = (bikes_df_cleaned == 'nan').any().any()

# Display the result
print(f"Do 'nan' values exist in the DataFrame? {unknown_values_exist}")

Do 'nan' values exist in the DataFrame? False


In [46]:
# Displaying concise information about the cleaned DataFrame
# This includes the number of non-null values, data types, and memory usage.
bikes_df_cleaned.info

<bound method DataFrame.info of           trip_id  trip_duration_seconds start_station_id  \
0        20148784                    840             7022   
1        20148785                    722             7399   
2        20148786                   1054             7269   
3        20148790                   1329             7721   
4        20148791                   1291             7721   
...           ...                    ...              ...   
5602106  26682295                   1516             7917   
5602107  26682655                    993             7013   
5602108  26660017                    511             7878   
5602109  26660153                     74             7918   
5602110  26660168                   3119             7918   

                      start_station_name start_time start_date end_station_id  \
0                 Simcoe St / Queen St W   00:00:00 2023-01-01           7703   
1           Lower Jarvis / Queens Quay E   00:01:00 2023-01-01           

In [47]:
# Merging the cleaned DataFrame bikes_df_cleaned with the stations DataFrame
# using 'start_station_id' from bikes_df_cleaned and 'station_id' from stations as the key.
# This performs a left join, retaining all rows from bikes_df_cleaned and matching them with station information from bikeshare_stations.
merged_df = bikes_df_cleaned.merge(bikeshare_stations, left_on='start_station_id', right_on='station_id', how='left')

# Display the updated DataFrame
merged_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type,station_id,name,lat,lon,capacity
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,1770,Casual Member,7022,Simcoe St / Queen St W,43.650,-79.387,39.000
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member,7399,Lower Jarvis St / Queens Quay E,43.644,-79.369,18.000
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655,-79.381,20.000
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,427,Casual Member,7721,Widmer St / King St W,43.646,-79.391,11.000
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,1840,Casual Member,7721,Widmer St / King St W,43.646,-79.391,11.000


In [48]:
# Displaying the column names of the merged DataFrame
merged_df.columns

Index(['trip_id', 'trip_duration_seconds', 'start_station_id',
       'start_station_name', 'start_time', 'start_date', 'end_station_id',
       'end_station_name', 'end_time', 'end_date', 'bike_id', 'user_type',
       'station_id', 'name', 'lat', 'lon', 'capacity'],
      dtype='object')

In [49]:
# Rename selected columns in the merged DataFrame for clarity
merged_df = merged_df.rename(columns={
    'physical_configuration': 'start_physical_configuration',
    'lat': 'start_lat',
    'lon': 'start_lon',
    'capacity': 'start_capacity',
    'is_charging_station': 'start_is_charging_station'
})

# Drop the 'station_id' column from the merged DataFrame
merged_df = merged_df.drop('station_id', axis=1)

In [50]:
#Inspecting the first five rows
merged_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type,name,start_lat,start_lon,start_capacity
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,1770,Casual Member,Simcoe St / Queen St W,43.650,-79.387,39.000
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member,Lower Jarvis St / Queens Quay E,43.644,-79.369,18.000
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member,Toronto Eaton Centre (Yonge St) - SMART,43.655,-79.381,20.000
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,427,Casual Member,Widmer St / King St W,43.646,-79.391,11.000
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,1840,Casual Member,Widmer St / King St W,43.646,-79.391,11.000


In [51]:
# Merge the merged DataFrame with bikeshare_stations using 'end_station_id' from merged_df 
# and 'station_id' from bikeshare_stations as the key.
# This performs a left join, retaining all rows from merged_df and matching them with end station information from bikeshare_stations.
bikes_cleaned_df = merged_df.merge(bikeshare_stations, left_on='end_station_id', right_on='station_id', how='left')

In [52]:
#Inspecting the first five rows of the merged dataframe
bikes_cleaned_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,...,user_type,name_x,start_lat,start_lon,start_capacity,station_id,name_y,lat,lon,capacity
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,...,Casual Member,Simcoe St / Queen St W,43.650,-79.387,39.000,7703,Oak St / Sackville St,43.661,-79.364,19.000
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,...,Casual Member,Lower Jarvis St / Queens Quay E,43.644,-79.369,18.000,7533,Housey St / Dan Leckie Way,43.638,-79.398,27.000
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,...,Annual Member,Toronto Eaton Centre (Yonge St) - SMART,43.655,-79.381,20.000,7076,York St / Queens Quay W,43.640,-79.380,57.000
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,...,Casual Member,Widmer St / King St W,43.646,-79.391,11.000,7685,King St W / Brant St,43.645,-79.397,22.000
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,...,Casual Member,Widmer St / King St W,43.646,-79.391,11.000,7685,King St W / Brant St,43.645,-79.397,22.000


In [53]:
# Renaming selected columns in the bikes_cleaned_df DataFrame for clarity regarding end stations
bikes_cleaned_df = bikes_cleaned_df.rename(columns={
    'physical_configuration': 'end_physical_configuration',
    'lat': 'end_lat',
    'lon': 'end_lon',
    'capacity': 'end_capacity',
    'is_charging_station': 'end_is_charging_station'
})

# Drop the 'station_id' column from the bikes_cleaned_df DataFrame
bikes_cleaned_df = bikes_cleaned_df.drop('station_id', axis=1)

In [54]:
#Inspecting the first five rows of the dataframe
bikes_cleaned_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,bike_id,user_type,name_x,start_lat,start_lon,start_capacity,name_y,end_lat,end_lon,end_capacity
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,1770,Casual Member,Simcoe St / Queen St W,43.650,-79.387,39.000,Oak St / Sackville St,43.661,-79.364,19.000
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,6400,Casual Member,Lower Jarvis St / Queens Quay E,43.644,-79.369,18.000,Housey St / Dan Leckie Way,43.638,-79.398,27.000
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,4068,Annual Member,Toronto Eaton Centre (Yonge St) - SMART,43.655,-79.381,20.000,York St / Queens Quay W,43.640,-79.380,57.000
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,427,Casual Member,Widmer St / King St W,43.646,-79.391,11.000,King St W / Brant St,43.645,-79.397,22.000
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,1840,Casual Member,Widmer St / King St W,43.646,-79.391,11.000,King St W / Brant St,43.645,-79.397,22.000


In [55]:
# Checking the data types of each column in the bikes_cleaned_df DataFrame
bikes_cleaned_df.dtypes

trip_id                          object
trip_duration_seconds             int64
start_station_id                 object
start_station_name               object
start_time                       object
start_date               datetime64[ns]
end_station_id                   object
end_station_name                 object
end_time                         object
end_date                 datetime64[ns]
bike_id                          object
user_type                        object
name_x                           object
start_lat                       float64
start_lon                       float64
start_capacity                  float64
name_y                           object
end_lat                         float64
end_lon                         float64
end_capacity                    float64
dtype: object

In [56]:
# Check and sum the missing values in each column of the bikes_cleaned_df DataFrame
bikes_cleaned_df.isna().sum()

trip_id                       0
trip_duration_seconds         0
start_station_id              0
start_station_name            0
start_time                    0
start_date                    0
end_station_id                0
end_station_name              0
end_time                      0
end_date                      0
bike_id                       0
user_type                     0
name_x                   129841
start_lat                129841
start_lon                129841
start_capacity           129841
name_y                   574877
end_lat                  574877
end_lon                  574877
end_capacity             574877
dtype: int64

# Merging the Dataframe to the Weather Dataset for Toronto for year 2023.

In [57]:
# Loading the weather dataset into the 'weather' dataframe
weather = pd.read_csv('/kaggle/input/weather-dataset/en_climate_daily_ON_6158355_2023_P1D(1).csv')

In [58]:
#Inspecting the first five rows
weather.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-79.400,43.670,TORONTO CITY,6158355,2023-01-01,2023,1,1,NaN,5.000,...,NaN,NaN,1.500,NaN,NaN,NaN,NaN,M,NaN,M
1,-79.400,43.670,TORONTO CITY,6158355,2023-01-02,2023,1,2,NaN,5.600,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,M,NaN,M
2,-79.400,43.670,TORONTO CITY,6158355,2023-01-03,2023,1,3,NaN,4.400,...,NaN,NaN,3.300,NaN,NaN,NaN,NaN,M,NaN,M
3,-79.400,43.670,TORONTO CITY,6158355,2023-01-04,2023,1,4,NaN,4.400,...,NaN,NaN,21.100,NaN,0.000,NaN,NaN,M,NaN,M
4,-79.400,43.670,TORONTO CITY,6158355,2023-01-05,2023,1,5,NaN,4.800,...,NaN,NaN,0.200,NaN,NaN,NaN,NaN,M,NaN,M


In [59]:
#Filtering the columns that are needed from the 'weather' dataframe
# Selecting specific columns
selected_columns = ['Date/Time', 'Mean Temp (°C)', 'Snow on Grnd (cm)', 'Total Precip (mm)']
weather = weather[selected_columns]

# Displaying the first five rows
weather.head()

,Date/Time,Mean Temp (°C),Snow on Grnd (cm),Total Precip (mm)
0,2023-01-01,3.900,NaN,1.500
1,2023-01-02,4.600,NaN,0.000
2,2023-01-03,3.600,NaN,3.300
3,2023-01-04,3.500,0.000,21.100
4,2023-01-05,4.000,NaN,0.200


In [60]:
#Checking the column datatypes
weather.dtypes

Date/Time             object
Mean Temp (°C)       float64
Snow on Grnd (cm)    float64
Total Precip (mm)    float64
dtype: object

In [61]:
#Renaming the columns
weather_df = weather.rename(columns={'Date/Time': 'date',\
                                        'Mean Temp (°C)': 'mean_temp(°C)',\
                                        'Snow on Grnd (cm)': 'snow_on_ground(cm)',\
                                        'Total Precip (mm)': 'total_precipitation(mm)'
                                       })

In [62]:
#Inspecting the changes
weather_df.head()

,date,mean_temp(°C),snow_on_ground(cm),total_precipitation(mm)
0,2023-01-01,3.900,NaN,1.500
1,2023-01-02,4.600,NaN,0.000
2,2023-01-03,3.600,NaN,3.300
3,2023-01-04,3.500,0.000,21.100
4,2023-01-05,4.000,NaN,0.200


In [63]:
# Converting the 'date' column to datetime format in the weather_df DataFrame
# The format='%Y-%m-%d' specifies the expected date format in the column.
# The errors='coerce' parameter replaces any parsing errors with NaT (Not a Time).
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d', errors='coerce')

In [64]:
#Fill missing values in the 'snow_on_ground(cm)' column of the weather_df DataFrame with a default value of 0.000.
weather_df['snow_on_ground(cm)'] = weather_df['snow_on_ground(cm)'].fillna(0.000)

In [65]:
#Inspecting the first 30 rows
weather_df.head(30)

,date,mean_temp(°C),snow_on_ground(cm),total_precipitation(mm)
0,2023-01-01,3.900,0.000,1.500
1,2023-01-02,4.600,0.000,0.000
2,2023-01-03,3.600,0.000,3.300
3,2023-01-04,3.500,0.000,21.100
4,2023-01-05,4.000,0.000,0.200
5,2023-01-06,4.000,0.000,0.000
6,2023-01-07,-0.500,0.000,0.000
7,2023-01-08,-3.100,0.000,0.000
8,2023-01-09,0.300,0.000,0.100
9,2023-01-10,-0.100,0.000,0.000


In [66]:
# Merging DataFrames based on the 'start_date' and 'date' columns
# This performs a left join, retaining all rows from bikes_cleaned_df and matching them with weather information from weather_df.
bikes_final_df = bikes_cleaned_df.merge(weather_df, left_on='start_date', right_on='date', how='left')

# Displaying the updated five rows of the DataFrame
bikes_final_df.head()


,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,...,start_lon,start_capacity,name_y,end_lat,end_lon,end_capacity,date,mean_temp(°C),snow_on_ground(cm),total_precipitation(mm)
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,...,-79.387,39.000,Oak St / Sackville St,43.661,-79.364,19.000,2023-01-01,3.900,0.000,1.500
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,...,-79.369,18.000,Housey St / Dan Leckie Way,43.638,-79.398,27.000,2023-01-01,3.900,0.000,1.500
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,...,-79.381,20.000,York St / Queens Quay W,43.640,-79.380,57.000,2023-01-01,3.900,0.000,1.500
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,...,-79.391,11.000,King St W / Brant St,43.645,-79.397,22.000,2023-01-01,3.900,0.000,1.500
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,...,-79.391,11.000,King St W / Brant St,43.645,-79.397,22.000,2023-01-01,3.900,0.000,1.500


In [67]:
# Drop the duplicated 'date' column from the weather information and columns 'name_x' and 'name_y'
bikes_final_df = bikes_final_df.drop('date', axis=1)
bikes_final_df = bikes_final_df.drop('name_y', axis=1)
bikes_final_df = bikes_final_df.drop('name_x', axis=1)

In [68]:
#Inspecting the dataframe after dropping the duplicate
bikes_final_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,...,user_type,start_lat,start_lon,start_capacity,end_lat,end_lon,end_capacity,mean_temp(°C),snow_on_ground(cm),total_precipitation(mm)
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,...,Casual Member,43.650,-79.387,39.000,43.661,-79.364,19.000,3.900,0.000,1.500
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,...,Casual Member,43.644,-79.369,18.000,43.638,-79.398,27.000,3.900,0.000,1.500
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,...,Annual Member,43.655,-79.381,20.000,43.640,-79.380,57.000,3.900,0.000,1.500
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,...,Casual Member,43.646,-79.391,11.000,43.645,-79.397,22.000,3.900,0.000,1.500
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,...,Casual Member,43.646,-79.391,11.000,43.645,-79.397,22.000,3.900,0.000,1.500


In [69]:
# Generate descriptive statistics for the bikes_final_df DataFrame
# The .describe() method calculates summary statistics for numerical columns.
# The .T transposes the result for better readability.
bikes_final_df.describe().T

,count,mean,min,25%,50%,75%,max,std
trip_duration_seconds,5602111.000,1077.739,60.000,423.000,689.000,1122.000,3980837.000,8658.879
start_date,5602111,2023-07-21 23:13:04.297989376,2023-01-01 00:00:00,2023-05-26 00:00:00,2023-07-27 00:00:00,2023-09-23 00:00:00,2023-12-31 00:00:00,NaN
end_date,5602111,2023-07-21 23:22:17.543151104,2023-01-01 00:00:00,2023-05-26 00:00:00,2023-07-27 00:00:00,2023-09-23 00:00:00,2024-01-07 00:00:00,NaN
start_lat,5472270.000,43.656,43.588,43.646,43.655,43.664,43.788,0.015
start_lon,5472270.000,-79.392,-79.553,-79.405,-79.389,-79.379,-79.124,0.031
start_capacity,5472270.000,21.485,7.000,15.000,19.000,24.000,63.000,8.902
end_lat,5027234.000,43.655,43.588,43.646,43.654,43.663,43.788,0.015
end_lon,5027234.000,-79.391,-79.553,-79.404,-79.388,-79.377,-79.124,0.032
end_capacity,5027234.000,21.607,7.000,15.000,19.000,24.000,63.000,9.148
mean_temp(°C),5587106.000,14.930,-14.900,8.500,17.800,20.900,26.900,7.896


In [70]:
bikes_final_df.isna().sum()

trip_id                         0
trip_duration_seconds           0
start_station_id                0
start_station_name              0
start_time                      0
start_date                      0
end_station_id                  0
end_station_name                0
end_time                        0
end_date                        0
bike_id                         0
user_type                       0
start_lat                  129841
start_lon                  129841
start_capacity             129841
end_lat                    574877
end_lon                    574877
end_capacity               574877
mean_temp(°C)               15005
snow_on_ground(cm)              0
total_precipitation(mm)     15005
dtype: int64

In [71]:
# Identifying rows where 'mean_temp(°C)' is missing
missing_mean_temp = bikes_final_df['mean_temp(°C)'].isnull()

# Display rows where 'mean_temp(°C)' is missing
missing_mean_temp_rows = bikes_final_df[missing_mean_temp]
print(missing_mean_temp_rows)

          trip_id  trip_duration_seconds start_station_id  \
5055838  26053508                    451             7067   
5055839  26053509                    607             7190   
5055840  26053510                    560             7046   
5055841  26053512                    771             7009   
5055842  26053514                    465             7536   
...           ...                    ...              ...   
5070838  26071050                    374             7762   
5070839  26071051                   1154             7548   
5070840  26071054                    579             7038   
5070841  26071057                    457             7386   
5070842  26071059                   1852             7506   

                          start_station_name start_time start_date  \
5055838                Yonge St / Harbour St   00:00:00 2023-11-07   
5055839           St. George St / Hoskin Ave   00:00:00 2023-11-07   
5055840           Niagara St / Richmond St W   00:00:00 2

Upon further investigation, there is a missing entry for 7th November, 2023 on the weather csv, so I looked up the average temperature of that day online and the precipitation which is 8.055 degrees and 0.000 mm precipitation.

In [72]:
# Replace NaN values in 'mean_temp(°C)' column with the number 8.055
bikes_final_df['mean_temp(°C)'] = bikes_final_df['mean_temp(°C)'].fillna(8.055)

# Replace NaN values in 'total_precipitation(mm)' column with the number 0.000
bikes_final_df['total_precipitation(mm)'] = bikes_final_df['total_precipitation(mm)'].fillna(0.000)

In [73]:
bikes_final_df.head()

,trip_id,trip_duration_seconds,start_station_id,start_station_name,start_time,start_date,end_station_id,end_station_name,end_time,end_date,...,user_type,start_lat,start_lon,start_capacity,end_lat,end_lon,end_capacity,mean_temp(°C),snow_on_ground(cm),total_precipitation(mm)
0,20148784,840,7022,Simcoe St / Queen St W,00:00:00,2023-01-01,7703,Oak St / Sackville St,00:14:00,2023-01-01,...,Casual Member,43.650,-79.387,39.000,43.661,-79.364,19.000,3.900,0.000,1.500
1,20148785,722,7399,Lower Jarvis / Queens Quay E,00:01:00,2023-01-01,7533,Housey St / Dan Leckie Way,00:13:02,2023-01-01,...,Casual Member,43.644,-79.369,18.000,43.638,-79.398,27.000,3.900,0.000,1.500
2,20148786,1054,7269,Toronto Eaton Centre (Yonge St),00:02:00,2023-01-01,7076,York St / Queens Quay W,00:19:34,2023-01-01,...,Annual Member,43.655,-79.381,20.000,43.640,-79.380,57.000,3.900,0.000,1.500
3,20148790,1329,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:26:09,2023-01-01,...,Casual Member,43.646,-79.391,11.000,43.645,-79.397,22.000,3.900,0.000,1.500
4,20148791,1291,7721,Widmer St / King St W,00:04:00,2023-01-01,7685,King St W / Brant St,00:25:31,2023-01-01,...,Casual Member,43.646,-79.391,11.000,43.645,-79.397,22.000,3.900,0.000,1.500


In [74]:
bikes_final_df.dtypes

trip_id                            object
trip_duration_seconds               int64
start_station_id                   object
start_station_name                 object
start_time                         object
start_date                 datetime64[ns]
end_station_id                     object
end_station_name                   object
end_time                           object
end_date                   datetime64[ns]
bike_id                            object
user_type                          object
start_lat                         float64
start_lon                         float64
start_capacity                    float64
end_lat                           float64
end_lon                           float64
end_capacity                      float64
mean_temp(°C)                     float64
snow_on_ground(cm)                float64
total_precipitation(mm)           float64
dtype: object

In [75]:
bikes_final_df.isna().sum()

trip_id                         0
trip_duration_seconds           0
start_station_id                0
start_station_name              0
start_time                      0
start_date                      0
end_station_id                  0
end_station_name                0
end_time                        0
end_date                        0
bike_id                         0
user_type                       0
start_lat                  129841
start_lon                  129841
start_capacity             129841
end_lat                    574877
end_lon                    574877
end_capacity               574877
mean_temp(°C)                   0
snow_on_ground(cm)              0
total_precipitation(mm)         0
dtype: int64

1. # Missing Value Imputation Technique: Mean

**Reasons for Choosing Mean:**

The mean is a straightforward statistical measure that is easy to understand, making it accessible for a wide audience, including non-technical stakeholders.

Imputing missing values with the mean helps maintain the central tendency or average of the data. This ensures that the imputed values align with the general trend of the observed values.

**Shortcomings of Mean Imputation:**

Imputing missing values with the mean assumes a normal distribution of the data. In cases where the distribution is skewed or non-normal, mean imputation may introduce bias.

Mean imputation does not account for the variability within the data. It assumes that all values are equally dispersed around the mean, which may not be true in datasets with diverse patterns.

**Considerations and Mitigation:**

While mean imputation has its limitations, it serves as a pragmatic choice in situations where the assumptions align with the data characteristics. Plus this was as a last resort after trying everything to get the correct data with no success.

In [76]:
# Mean Imputation
mean_imputed_df = bikes_final_df.copy()

# Replace missing values with mean for the specified columns
mean_imputed_df['start_lat'] = mean_imputed_df['start_lat'].fillna(mean_imputed_df['start_lat'].mean())
mean_imputed_df['start_lon'] = mean_imputed_df['start_lon'].fillna(mean_imputed_df['start_lon'].mean())
mean_imputed_df['start_capacity'] = mean_imputed_df['start_capacity'].fillna(mean_imputed_df['start_capacity'].mean())
mean_imputed_df['end_lat'] = mean_imputed_df['end_lat'].fillna(mean_imputed_df['end_lat'].mean())
mean_imputed_df['end_lon'] = mean_imputed_df['end_lon'].fillna(mean_imputed_df['end_lon'].mean())
mean_imputed_df['end_capacity'] = mean_imputed_df['end_capacity'].fillna(mean_imputed_df['end_capacity'].mean())


In [77]:
#Rounding down the 'start_capacity' and 'end_capacity'
mean_imputed_df['start_capacity'] = np.floor(mean_imputed_df['start_capacity'])
mean_imputed_df['end_capacity'] = np.floor(mean_imputed_df['end_capacity'])

In [78]:
#Confirmation of whether there are any more missing values
mean_imputed_df.isna().sum()

trip_id                    0
trip_duration_seconds      0
start_station_id           0
start_station_name         0
start_time                 0
start_date                 0
end_station_id             0
end_station_name           0
end_time                   0
end_date                   0
bike_id                    0
user_type                  0
start_lat                  0
start_lon                  0
start_capacity             0
end_lat                    0
end_lon                    0
end_capacity               0
mean_temp(°C)              0
snow_on_ground(cm)         0
total_precipitation(mm)    0
dtype: int64

In [79]:
#Summary Statistics of the clean data.
mean_imputed_df.describe().T

,count,mean,min,25%,50%,75%,max,std
trip_duration_seconds,5602111.000,1077.739,60.000,423.000,689.000,1122.000,3980837.000,8658.879
start_date,5602111,2023-07-21 23:13:04.297989376,2023-01-01 00:00:00,2023-05-26 00:00:00,2023-07-27 00:00:00,2023-09-23 00:00:00,2023-12-31 00:00:00,NaN
end_date,5602111,2023-07-21 23:22:17.543151104,2023-01-01 00:00:00,2023-05-26 00:00:00,2023-07-27 00:00:00,2023-09-23 00:00:00,2024-01-07 00:00:00,NaN
start_lat,5602111.000,43.656,43.588,43.646,43.656,43.664,43.788,0.015
start_lon,5602111.000,-79.392,-79.553,-79.404,-79.389,-79.379,-79.124,0.031
start_capacity,5602111.000,21.474,7.000,15.000,19.000,24.000,63.000,8.798
end_lat,5602111.000,43.655,43.588,43.646,43.655,43.662,43.788,0.014
end_lon,5602111.000,-79.391,-79.553,-79.402,-79.390,-79.379,-79.124,0.030
end_capacity,5602111.000,21.544,7.000,15.000,19.000,23.000,63.000,8.667
mean_temp(°C),5602111.000,14.912,-14.900,8.500,17.800,20.900,26.900,7.894


**Conclusion:**

After meticulously cleaning and integrating the datasets encompassing bikeshare data, weather conditions, and station dynamics, we've laid a solid foundation for extracting meaningful insights into urban mobility trends. By ensuring the accuracy and coherence of our data, we've positioned ourselves to derive actionable conclusions that can inform strategic decision-making and optimize the efficiency of bikeshare systems.